# Approach 1: The "DIY" Toolsmith - JSON Schema Style!

### Step 1: Crafting Your Tool Description in JSON Schema

In [1]:
multiply_tool_schema = {

        "name": "multiply_numbers",
        "description": "Multiplies two numbers together.",
        "parameters": {
            "type": "object",
            "properties": {
                "num1": {
                    "type": "integer",
                    "description": "The first number to multiply."
                },
                "num2": {
                    "type": "integer",
                    "description": "The second number to multiply."
                }
            },
            "required": ["num1", "num2"],
        },
    }
tools_schema = [multiply_tool_schema] # We'll have a list of tools (even if just one for now)

print(tools_schema) # Let's peek at our tool schema!

[{'name': 'multiply_numbers', 'description': 'Multiplies two numbers together.', 'parameters': {'type': 'object', 'properties': {'num1': {'type': 'integer', 'description': 'The first number to multiply.'}, 'num2': {'type': 'integer', 'description': 'The second number to multiply.'}}, 'required': ['num1', 'num2']}}]


### Step 2: Setting up Gemini (and a Function to Execute the Tool)

In [2]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv("D:\\Code\\AI\\.env") # Load API key from .env file (best practice!)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

# --- Function to actually multiply ---
def multiply_numbers_function(num1, num2):
    """This is the Python function that actually multiplies."""
    return num1 * num2

d:\Code\ENVS\llmenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 3: Prompting Gemini to Use the Tool!

In [8]:
user_prompt = "What is 15 times 6?"

response = model.generate_content(
    [user_prompt],
    tools=[tools_schema]  #  <---  Crucially, we pass our tool schema here!]
)

print(response) # Let's see what Gemini initially says (might just be text)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "multiply_numbers",
                  "args": {
                    "num2": 6.0,
                    "num1": 15.0
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": 5.078285799494811e-06
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 38,
        "candidates_token_count": 7,
        "total_token_count": 45
      },
      "model_version": "gemini-2.0-flash"
    }),
)


### Step 4: Handling Gemini's Response - Tool Call or Direct Answer? & Step 5: Sending the Tool Result Back to Gemini (Crucial Step!)

In [9]:
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    function_name = function_call.name
    function_args = function_call.args

    print(f"Gemini wants to call function: {function_name}") # Debugging!
    print(f"With arguments: {function_args}") # Debugging!

    if function_name == "multiply_numbers": # Check if it's our multiply tool
        num1 = function_args.get("num1") # Extract parameters
        num2 = function_args.get("num2")

        # ---  Call our Python function to EXECUTE the tool! ---
        tool_result = multiply_numbers_function(num1, num2)
        print(f"Tool execution result: {tool_result}") # Debugging!

        # ---  Now we need to send the tool result back to Gemini! ---
        # ---  Prepare the tool result to send back to Gemini ---
        tool_response_content = {
            "function_response": {
                "name": function_name,
                "response": { "result": tool_result } #  Wrap the result nicely!
            }
        }

        # ---  Send it BACK to Gemini in a NEW generate_content call! ---
        response = model.generate_content(
            [user_prompt, tool_response_content] #  Include original prompt AND tool response!
        )

        print("Final Gemini response AFTER tool use:")
        print(response.text)

else: # Gemini can answer directly
    print("Gemini answered directly:")
    print(response.text)

Gemini wants to call function: multiply_numbers
With arguments: <proto.marshal.collections.maps.MapComposite object at 0x000001708CE46840>
Tool execution result: 90.0
Final Gemini response AFTER tool use:
15 times 6 is 90.



# Approach 2: "@tool " Magic!- Langchain's

### Step 1: Define Your Tool with @tool (Python Magic!)

In [10]:
print("LANGCHAIN APPROACH")
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

@tool
def multiply_tool(num1: int, num2: int) -> int:
    """Multiply two numbers together."""
    return num1 * num2

print(multiply_tool.args_schema.model_json_schema()) # get the same schema details as above
# Let's inspect some of the attributes associated with the tool.
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)
print(multiply_tool.return_direct)

# Tool creation
tools = [multiply_tool] # Langchain tools are just Python functions decorated with @tool!

print(tools) #  Langchain understands this is a tool!

LANGCHAIN APPROACH
{'description': 'Multiply two numbers together.', 'properties': {'num1': {'title': 'Num1', 'type': 'integer'}, 'num2': {'title': 'Num2', 'type': 'integer'}}, 'required': ['num1', 'num2'], 'title': 'multiply_tool', 'type': 'object'}
multiply_tool
Multiply two numbers together.
{'num1': {'title': 'Num1', 'type': 'integer'}, 'num2': {'title': 'Num2', 'type': 'integer'}}
False
[StructuredTool(name='multiply_tool', description='Multiply two numbers together.', args_schema=<class 'langchain_core.utils.pydantic.multiply_tool'>, func=<function multiply_tool at 0x000001708DC0DD00>)]


### Step 2: Setting Up Gemini with Langchain - Even Simpler!

In [11]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=os.getenv("GOOGLE_API_KEY"))

### Step 3: Tool Binding - Connecting Tools to the Model (Langchain Magic!)

In [12]:
# Tool binding
model_with_tools = llm.bind_tools(tools)

### Step 4: Run the Tool-Enabled Model! (Effortless Tool Calling)

In [13]:
user_question = "What is 15 times 14 ?"
# Tool calling
response = model_with_tools.invoke(user_question)

# model_with_tools.invoke(response) #  You could even pass the previous response back in for more complex flows

print("\n RESPONSE \n",response) #  The final answer!
print(response.tool_calls) #  See the details of the tool call!


 RESPONSE 
 content='' additional_kwargs={'function_call': {'name': 'multiply_tool', 'arguments': '{"num2": 14.0, "num1": 15.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-a883b04f-008d-4485-bfc5-bc5f097d93cc-0' tool_calls=[{'name': 'multiply_tool', 'args': {'num2': 14.0, 'num1': 15.0}, 'id': 'abd7244e-a87c-4e87-a4fb-83833c35118f', 'type': 'tool_call'}] usage_metadata={'input_tokens': 26, 'output_tokens': 7, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}}
[{'name': 'multiply_tool', 'args': {'num2': 14.0, 'num1': 15.0}, 'id': 'abd7244e-a87c-4e87-a4fb-83833c35118f', 'type': 'tool_call'}]


### Step 5:  Taking it Further - Conversational Tool Calling!

In [14]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(user_question)]
messages.append(response)

for tool_call in response.tool_calls:
    selected_tool = { "multiply_tool": multiply_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)


response2 =model_with_tools.invoke(messages)

print("\n RESPONSE2 \n",response2.content) 


 RESPONSE2 
 15 times 14 is 210.
